---
Material for the course "Photonics systems and devices"

The following notes written by SERGIO G RODRIGO (2021-2022)</font> 

---

- *Use google colab to see the Notebook index*
- *See classes and functions/methods descriptions calling name_class.__doc__; name_method.__doc__*
- *Notation and equations from "Optical Waveguide Theory", A.W. Snyder and
J. Love Springer, Boston, MA (1983) 1st Ed https://doi.org/10.1007/978-1-4613-2813-1*
- The chapters, pages, figures and tables of Snyder&Love are used to describe optical fiber parameters, field definitions, dispersion relation definitions...

# Description

+ Solver of the eigenvalue equations for the step-profile fiber from Snyder&Love, Section 12-9 Table 12-4. 

# Google access

In [ ]:
#Your google drive is made accesible to Colab.
try:
    from google.colab import drive 
    drive.mount('/content/drive/')
    # The working directory
    %cd "/content/drive/MyDrive/optical_fiber"
    %ls -lht 
    # To import own packages set local path in packages syspath
    import sys
    sys.path.insert(0,"./")
except ImportError:
    print("You are not in google.colab!!")    
    pass

# Physics constants


In [ ]:
c0= 299792458.0 #m/s
eps0= 8.854187817e-12    #F·m-1 
mu0= 1.0/(eps0*(c0**2)) #N·A-2
con_wltoeV=1239.828330

# Auxiliary functions for plotting

In [ ]:
from OFiber_plot import step_array_to_regular

# Auxiliary functions for finding EM mode solutions

In [ ]:
from OFiber_find import find_EM_modes,\
                        find_EM_modes_loop

# Optical fiber Class (step-profile)

In [ ]:
from OFiber_class import OFiber

#Step-profile optical fiber 

+ *NOTE*: to reproduce Snyder&Love results run just one of the following options at time. Otherwise you have to modify the parameters defining the optical fiber in a new notebook cell.
+ The mode labels as generated by the notebook do not corresponds to the usual notation for exact modes ($HE_{11}$...) or weakly guiding aproximation labels ($LP_{01}$...). 
+ The modes are generated in the order and labelled as follows:
 - 1- Hybrid modes 2- TE modes 3- TM modes
 - 2- Each column labels a mode as 'U-nu'+str(order)+'m'+str(m+1). Where *nu*=order of the Bessel function and *m* is the order of appearance of the zero value in the trascendental equation (dispersion relation identity). 

### Option 1: high index contrast waveguide (Fig. 12-4 Snyder&Love). Definition of the OFiber through $n_{co}$,$n_{cl}$, $\Delta$, $\rho$ 

In [ ]:
'''
Radious has been chosen so the wavelength range of operation 
for both waveguides will be the same. Therefore:
rhow=of.rho*np.sqrt(of.nco**2-of.ncl**2)/np.sqrt(ofw.nco**2-ofw.ncl**2)
print("rho for weakly guinding from high index waveguide",rhow)
'''

'''
High index contrast waveguides
(see Fig. 12-4 Snyder&Love)
'''
rho=0.2 #micras
nco=2.5
delta=0.32  
of=OFiber(rho,nco=nco,delta=delta)
# Dispersion relation V vs U file 
file_name = 'fig12-4_Snyder_Love_new.csv'   
print(of)

### Option 2: weakly guiding waveguide (14-4 Snyder&Love). Definition of the OFiber through $n_{co}$,$n_{cl}$, $\Delta$, $\rho$ 

In [ ]:
'''
Weakly guiding waveguides
(see Fig. 14-4 Snyder&Love)
'''
rho=2.4226 #micras
ncl=1.45
delta=0.0064
ofw=OFiber(rho,ncl=ncl,delta=delta)
# Dispersion relation V vs U file 
file_name = 'weak_fig14-4_Snyder_Love_new.csv'    
print(of)

## Dispersion relation calculations

#### Introduce $V$ and $\nu$ (mode order)

In [ ]:
import numpy as np
order_exact=1
order_weak=0
Vi=1.0
Vf=5.1
V_step=100
Vloop=np.linspace(Vi,Vf,V_step)
V=min(Vloop)

# Args for find_EM_modes() and find_EM_modes_loop()
Uini=0.001 # U_fin = Vmax - tolerance, tolerance=0.001
steps=10e3
args_exact = (V,order_exact)
args_exact_loop={"Vloop":Vloop,
                  "order":order_exact,
                  "ofiber":of,
                 }
args_weak = (V,order_weak)
args_weak_loop={"Vloop":Vloop,
                  "order":order_weak,
                  "ofiber":of}          
 


####  Calculate exact dispersion relation 
$U$ vs $V$
+ Search for $U$ at given $V$ 
+ Regular approach from Snyder and Love "Optical Waveguide Theory" Section 12-9 Table 12-4

In [ ]:
import pandas as pd

Uexact_hybrid=[]
for i in range(1,5): #order=0 coincides with TE and TM modes
    args_exact_loop['order']=i
    Uexact_hybrid.append(find_EM_modes_loop(of.em_modes('hybrid'),args_exact_loop,Uini,steps,verbose=True))
Uexact_TE=find_EM_modes_loop(of.em_modes('TE'),args_exact_loop,Uini,steps,verbose=True)
Uexact_TM=find_EM_modes_loop(of.em_modes('TM'),args_exact_loop,Uini,steps,verbose=True)


# Save to a .csv file with Pandas U vs V
Uexact_hybrid_expand=[]
for i in range(0,4):#range(len(Uexact_hybrid)):         
    Uexact_hybrid_expand.append(step_array_to_regular(Uexact_hybrid[i]))                
#Uexact_hybrid_expand=[]
Uexact_hybrid_expand.append(step_array_to_regular(Uexact_TE))
Uexact_hybrid_expand.append(step_array_to_regular(Uexact_TM))
df=pd.DataFrame({"V": Vloop})    
for order in range(len(Uexact_hybrid_expand)):   
  vlen=Uexact_hybrid_expand[order].shape[0]
  clen=Uexact_hybrid_expand[order].shape[1]
  #print(vlen,clen)
  for m in range(vlen):                 
    df['U-nu'+str(order)+'m'+str(m+1)] = Uexact_hybrid_expand[order][m]        
# saving the excel
df.to_csv(file_name,sep=';',index=False)
print('DataFrame is written to Excel File successfully.')

####  Calculate exact dispersion relation (alternative)


In [ ]:
if(False):
  Uexact_hybrid_alt=find_EM_modes_loop(of.em_modes('hybrid-alt'),args_exact_loop,Uini,steps,verbose=True)

####  Calculate weakly guiding approximation dispersion relation for $\nu=0$

In [ ]:
if(False):
  Uweak=find_EM_modes_loop(of.em_modes('weak'),args_weak_loop,Uini,steps,verbose=True)
  Uweak_expand=step_array_to_regular(Uweak)

#### Check a single (U,V) case

In [ ]:
print(of)
V=6.91573
order=0
args=(V,order)
Uini=0.001
Ufin=9.999
resolution=(Ufin-Uini)/10e3
find_EM_modes(of.em_modes('TM'),args,Uini,Ufin,resolution,verbose=True) 
print("Eigenvalue equation value=",of.em_modes('TM')(U=6.91573,V=6.8233,nu=0))